# Course 1: Lidar
## Part 2: Point Cloud Segmentation 
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Use the [Point Cloud Library](https://pointclouds.org/) (PCL) to segment point clouds;
* Implement the [Random Sample Consensus](https://en.wikipedia.org/wiki/Random_sample_consensus) (RANSAC) algorithm for planar model fitting;
* Perform ground plane separation to "remove" the road surface from our point cloud data.

## 1. Introduction

### 1.1. Point Cloud Segmentation

Our goal in this lesson is to _process_ the point cloud data we received from the LiDAR sensor and _segment_ it in such a way that we are able to "remove" any of the points from our scene which do not belong to the obstacles we wish to detect. In the driving domain, one common type of points in which we wish to remove are _ground plane_ points. These are the points which "belong" to the road surface(s) that the vehicle(s) travel along. 

There are many potential use cases for LiDAR sensors in the automotive space, with point cloud segmentation being one such use case discussed in detail here. There are many other uses for LiDAR sensors and point clouds in the automotive and robotics space: multi-modal sensor calibration / fusion techniques, semi-supervised deep learning models for depth estimation, and for bench-marking tasks used to e.g., evaluate the performance of a "stand-alone" sensor on a given task (such as RGB camera-based vision systems on the monocular depth estimation task).

In the following section, we will narrow our scope to focus only on ground plane estimation as it relates to point cloud segmentation in the section below...

#### Ground Plane Estimation: Problem Setup

In this subsection we will review the topic of ground plane estimation, which is a preliminary exercise for point cloud segmentation and is essential to performing obstacle detection on LiDAR point cloud data. Ground plane estimation involves "detecting" and segmenting points in a point cloud which belong to the ground surface(s) in a scene. Using a technique known as planar fitting, we can extract the points from a levelled surface and remove them from the point cloud. Once these points are effectively discarded, we can begin to segment the object(s) or obstacle(s) for upstream tasks such as detection, tracking, classification, etc.

One primary assumption we will make about the surface we wish to segment is that it is level. In other words, we assume the ground plane / road surface our ego-vehicle is travelling on is _non-inclined_, i.e., that it has a $0^{\circ}$ elevation coinciding with the normal vector (usually defined as the vector perpendicular to the road surface; the upward-facing $z$-axis). To further simplify this problem, we also assume that the surface the ego-vehicle is travelling on is a uniform [planar surface](https://en.wikipedia.org/wiki/Planar_Riemann_surface). In other words, we assume there are no imperfections or deviations along the surface, which would cause the surface to deviate from a smooth, continuous composition. This implies that a constant elevation and constant slope exist along the surface's length and width. In the real-world, road surfaces have elevation, slope, and [textures](https://en.wikipedia.org/wiki/Road_texture) — design elements which break the simplifications we are forming for this toy problem. While real-world road surfaces complicate our rudimentary approach to road plane segmentation, advancements in R&D have been able to handle the non-uniform geometries that exist with non-planar road surfaces by employing various techniques such as deep learning feature extraction [1].   

In our toy example, we proceed with a planar surface assumption that will allow us to form a set of geometric constraints onto our environment which will help us better project a "surface" used to blanket the potential LiDAR points we wish to include in the segmented point cloud. Such a simplification will allow us to reduce the plane fitting estimation into an exercise of "best-fit". Using the [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm, we can "work out" which points belong to the ground plane / simplified road surface by randomly-sampling a small set of points, fitting a "plane equation" to those sampled points, then performing an iterative two-step inlier "verification" and "consensus" stage to refine the planar surface estimation. It is with this iterative least-squares estimation algorithm we can determine a ground plane equation which "fits" or includes the most inlier (true positive) points, leaving us with a "segmentation" of sorts that includes most, if not all, the points belonging to our modelled road surface.

## 2. Programming Task

## 3. Closing Remarks

#### Alternatives


#### Extensions of task

## 4. Future Work

* ⬜️ ;
* ⬜️ .

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] Paigwar, A. Erkent, Ö., Sierra-Gonzalez, D. Laugier, C. GndNet: Fast Ground Plane Estimation and Point Cloud Segmentation for Autonomous Vehicles. IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS). 2020, pp. 2150-2156.


Helpful resources:
* 